### Startup imports

In [1]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.slurm_manager import lsf_manager
from autoqchem.helper_classes import lsf_status

import logging
logging.basicConfig(level=logging.INFO)

### Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [2]:
sm=slurm_manager(user='zuranski', host='adroit.princeton.edu')

### Provide some smiles string(s)

and copy into them into ```my_smiles``` variable. It's a list of strings (a Python list can be of any size)

In [3]:
my_smiles=[
    "C(C1C(C(C(C(O1)O)O)O)O)O",
    "c1ccc(F)cc1",
]

For a list of smiles, you can also read them out from a csv file in this way, assumed file name ```smiles_strings.csv```, also assuming the strings are in the first column, and there are no headers

In [4]:
# my_smiles = pd.read_csv("smiles_strings.csv", header=None, usecols=[0])[0].tolist()
# my_smiles

### Initialize the molecules and generate conformations

We will use some ```for``` loops to loop over the smiles

In [4]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, max_num_conformers=3))

INFO:autoqchem.molecule:Initializing molecule with canonical smiles: OCC1OC(O)C(C(C1O)O)O
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Conformer Search generated 3 conformations of OCC1OC(O)C(C(C1O)O)O molecule
INFO:autoqchem.molecule:Initializing molecule with canonical smiles: Fc1ccccc1
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Conformer Search generated 1 conformations of Fc1ccccc1 molecule


Draw some molecules

In [5]:
mols[0].draw(2)

### Create gaussian jobs for each conformation

In [7]:
for mol in mols:
    lsf.create_jobs_for_molecule(mol, workflow_type="equilibrium")

INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 3 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


In [8]:
lsf.get_job_stats(split_by_can=True)

status,created
can,
Fc1ccccc1,1
OCC1OC(O)C(C(C1O)O)O,3


### Submit the jobs to the server 

if you are not connected, it will ask for password + Duo

In [6]:
lsf.remove_jobs(lsf.get_jobs())

INFO:autoqchem.slurm_manager:Creating connection to euler.ethz.ch as jgoetz
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)




Password: ········


INFO:paramiko.transport:Authentication (keyboard-interactive) successful!


INFO:autoqchem.slurm_manager:Connected to euler.ethz.ch as jgoetz.


In [9]:
lsf.submit_jobs()

INFO:autoqchem.slurm_manager:Submitting 4 jobs.
INFO:paramiko.transport.sftp:[chan 9] Opened sftp connection (server version 3)


UnexpectedExit: Encountered a bad command exit code!

Command: 'cd /cluster/scratch/jgoetz/gaussian && bsub < /cluster/scratch/jgoetz/gaussian/C6H12O6_ed82_conf_0.sh'

Exit code: 255

Stdout:



Stderr:

Gaussian job.
Your environment is not configured for Gaussian.
Please execute 'module load gaussian' and try again.
Request aborted by esub. Job not submitted.



Check the status of the jobs, locally and remotely

In [14]:
lsf.get_job_stats(split_by_can=True)

status
can


In [11]:
lsf.bjobs(summary=False)

INFO:autoqchem.slurm_manager:Creating connection to euler.ethz.ch as jgoetz
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)




Password: ········


INFO:paramiko.transport:Authentication (keyboard-interactive) successful!


INFO:autoqchem.slurm_manager:Connected to euler.ethz.ch as jgoetz.


<Result cmd='bjobs -u jgoetz' exited=0>

### Retrieve jobs

If there are some finished jobs the log files will be checked for completion and downloaded locally.

In [21]:
lsf.retrieve_jobs()

INFO:autoqchem.slurm_manager:There are 0 running/pending jobs, 4 finished jobs.
INFO:autoqchem.slurm_manager:Retrieving log files of finished jobs.
INFO:autoqchem.slurm_manager:0 jobs finished successfully (all Gaussian steps finished normally). 4 jobs failed.


### Resubmit incomplete jobs

In [14]:
lsf.resubmit_incomplete_jobs()

INFO:autoqchem.slurm_manager:Substituting last checked geometry in the new input file.


AttributeError: 'NoneType' object has no attribute 'group'

### Upload molecules to the database

In [ ]:
sm.upload_done_molecules_to_db(tag="tutorial_AMZ")